# Generate log2CPM for CMC data

In [1]:
library(tidyverse)
library(synapser)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


New synapser version detected: 
    You are using synapser version 0.9.77.
    synapser version 0.10.101 is detected at http://ran.synapse.org.
    To upgrade to the latest version of synapser, please run the following command:
    install.packages("synapser", repos="http://ran.synapse.org")



TERMS OF USE NOTICE:
  When using Synapse, remember that the terms and conditions of use require that you:
  1) Attribute data contributors when discussing these data or results from these data.
  2) Not discriminate, identify, or recontact individuals or groups represented by the data.
  3) Use and contribute only 

In [2]:
synLogin()

Welcome, kj.benjamin!

NULL

## Phenotypes

In [3]:
# Download clinical metadata 
CLINICAL_ID = 'syn3354385'
clinical = data.table::fread(synGet(CLINICAL_ID, version = 4)$path)

# Download RNASeq metadata
METADATA_QC_DLPFC_ID = 'syn18358379' 
metadata = data.table::fread(synGet(METADATA_QC_DLPFC_ID, version = 3)$path)

# Join clinical and RNASeq metadata 
md = right_join(clinical, metadata, by = c("Individual ID" = "Individual_ID")) %>% 
    mutate(Dx = fct_recode(Dx, AFF_BP = "BP", AFF_BP = "AFF", Other = "undetermined", 
                           Control = "Control", SCZ = "SCZ")) %>%
    filter(Dx %in% c("Control", "SCZ"), Sex %in% c('XX', 'XY'))

# Compute read pair metrics and add Institution-Dx variable
md <- md %>%
  mutate(MappedRead_Pairs = Mapped_Reads/2) %>%
  mutate(`Institution-Dx` = paste0(`Institution`, "-", `Dx`)) %>% 
  mutate(TotalRead_Pairs = Total_Reads/2)

# Add MDS from SNPs
mds_file = paste0('/ceph/users/jbenja13/projects/sex_sz_ria/input/commonMind/',
                  'genotypes/mds/_m/CMC_MSSM-Penn-Pitt_DLPFC_QC.mds')
mds = data.table::fread(mds_file)
colnames(mds) = gsub('C', 'snpPC', colnames(mds))

pheno_file = paste0('/ceph/users/jbenja13/projects/sex_sz_ria/input/commonMind/', 
                    'phenotypes/combine_files/_m/CMC_phenotypes_all.csv')
pheno = read.csv(pheno_file, stringsAsFactors = F) 
genotypes = merge(pheno, mds, by.y='IID', by.x='Genotypes.Genotyping_Sample_ID') 

genotypes = genotypes %>% 
    dplyr::select("Individual_ID", starts_with("snpPC")) %>%
    rename("Individual ID"=Individual_ID)

md = md %>% left_join(genotypes, by="Individual ID") %>% distinct

md %>% dim

[1] 858  65

In [4]:
phenotypes = md %>% select("SampleID", "Individual ID", "Institution", 
                           "Reported Gender", "Dx", "Age of Death") %>%
    mutate(`Age of Death` = ifelse(`Age of Death` == "90+", "90", `Age of Death`))
colnames(phenotypes) <- gsub(' ', '_', colnames(phenotypes))
phenotypes %>% data.table::fwrite("cmc_phenotypes.csv", sep=',')
phenotypes %>% head(2)

SampleID,Individual_ID,Institution,Reported_Gender,Dx,Age_of_Death
<chr>,<chr>,<chr>,<chr>,<fct>,<chr>
MSSM_RNA_PFC_155,CMC_MSSM_087,MSSM,Female,Control,90
MSSM_RNA_PFC_280,CMC_MSSM_226,MSSM,Female,Control,90


## Gene expression (counts)

### Combined counts

In [5]:
# Download counts (DLPFC - MSSM)
COUNT_ID = 'syn17346208'
count = data.table::fread(synGet(COUNT_ID, version=2)$path) #synapser has updated without backwards compatibility
count$transcript_id.s. = NULL

# Download gene lengths (DLPFC - MSSM)
genelen_CMC = data.table::fread(synGet('syn17346397', version = 2)$path) %>%
    gather(sampleID, Length, -gene_id, -`transcript_id(s)`) %>%
    group_by(gene_id) %>%
    summarise(Length = median(Length, na.rm = T)) %>%
    ungroup() %>% data.frame()

 # Download counts (DLPFC - HBCC)
COUNT_ID = 'syn17894685'
count_HBCC = data.table::fread(synGet(COUNT_ID, version = 4)$path)
count_HBCC$transcript_id.s. = NULL

# Join HBCC and MSSM counts
NEW.COUNTS = full_join(count, count_HBCC, by = c("gene_id")) %>%
    column_to_rownames(var='gene_id') %>% t %>%
    as.data.frame %>% rownames_to_column %>% 
    filter(rowname %in% md$SampleID) %>% 
    column_to_rownames(var="rowname") %>% t %>%
    as.data.frame
NEW.COUNTS[1:2, 1:5]

,MSSM_RNA_PFC_1,MSSM_RNA_PFC_2,MSSM_RNA_PFC_3,MSSM_RNA_PFC_4,MSSM_RNA_PFC_5
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003.14,124,103,160,366,268
ENSG00000000005.5,1,0,1,4,2


In [6]:
NEW.COUNTS %>% dim

[1] 58347   858

### CPM transformation and save

In [7]:
edgeR::cpm(NEW.COUNTS, log=TRUE) %>% as.data.frame %>% 
    rownames_to_column %>% rename("Geneid"="rowname") %>%
    data.table::fwrite("cmc_log2cpm.tsv", sep='\t')

## Gene annotation

In [8]:
# Get background genes 
backgroundGenes = data.frame(gene_id = rownames(NEW.COUNTS)) %>%
  mutate(id = gene_id) %>%
  separate(id, c('ensembl_gene_id','position'), sep = '\\.')

# Define biomart object
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                host = "uswest.ensembl.org", # Ensembl Release 99 (January 2020)
                dataset = "hsapiens_gene_ensembl")
# Query biomart
Ensemble2HGNC <- biomaRt::getBM(attributes = c("ensembl_gene_id", "hgnc_symbol", 
                                      "percentage_gene_gc_content", "gene_biotype", 
                                      "chromosome_name"),
                       filters = "ensembl_gene_id", 
                       values = backgroundGenes$ensembl_gene_id,
                       mart = mart)

In [9]:
backgroundGenes %>% 
    inner_join(Ensemble2HGNC, by=c("ensembl_gene_id")) %>% 
    select(-c(percentage_gene_gc_content, gene_biotype, position)) %>% 
    data.table::fwrite("cmc_gene_annotation.tsv", sep='\t')

## Repreducibility Information

In [10]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()

[1] "2021-07-10 10:12:35 EDT"

   user  system elapsed 
 56.632   6.197 153.661 

─ Session info ───────────────────────────────────────────────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.3 (2020-10-10)
 os       Arch Linux                  
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       America/New_York            
 date     2021-07-10                  

─ Packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────
 package        * version  date       lib source        
 AnnotationDbi    1.52.0   2020-10-27 [1] Bioconductor  
 askpass          1.1      2019-01-13 [1] CRAN (R 4.0.2)
 assertthat       0.2.1    2019-03-21 [1] CRAN (R 4.0.2)
 backports        1.2.1    2020-12-09 [1] CRAN (R 4.0.2)
 base64enc        0.1-3    2015-07-28 [1] CRAN (R 4.0.2)
 Biobase          2.50.0   20